In [1]:
# Created by Alexander Bolejack & Austin Roddy
# Illinois Institute of Technology
# Intllgnc Txt Analys Knwldg Mgm (CS-481-01), Spring 2022
# Programming Assignment 2 - Final Project

In [2]:
# Inport the necessary libraries
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords as sw
import re

In [3]:
# dowload corpori for nltk (used later)
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/austin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Import the data, and drop the unnecessary columns, remove non-alphabetical characters(first pass)
full_dataset_df = pd.read_csv ("dataset.csv")
full_dataset_df.drop('app_id', inplace=True, axis=1)
full_dataset_df.drop('app_name', inplace=True, axis=1)
full_dataset_df.drop('review_votes', inplace=True, axis=1)
full_dataset_df.rename(columns={'review_text': 'text', 'review_score': 'label'}, inplace=True)

In [5]:
# Remove duplicate messages in entries in the dataset
duplicates = full_dataset_df.duplicated(subset='text', keep='first')
full_dataset_df = full_dataset_df.drop(full_dataset_df[duplicates].index)


# I wouldnt trust this function HAHAHAHA it deletes 1.6 million rows

In [6]:
# Normalize the labels if -1 switch to 0
full_dataset_df['label'] = full_dataset_df['label'].replace(-1, 0)

In [7]:
# Randomize the dataframe
randomized_full_dataset_df = full_dataset_df.sample(frac=1).reset_index(drop=True)

In [8]:
# Split the data into training and testing sets (90% training, 10% testing) (and randomize... again...? *shrug*)
training_dataset_df = randomized_full_dataset_df.sample(frac=0.9,random_state=200)
testing_dataset_df = randomized_full_dataset_df.drop(training_dataset_df.index)

In [9]:
# Create both pos and neg versions of training and testing sets (4 sets)
training_pos_dataset_df = training_dataset_df[training_dataset_df['label'] == 1]
training_neg_dataset_df = training_dataset_df[training_dataset_df['label'] == 0]
testing_pos_dataset_df = testing_dataset_df[testing_dataset_df['label'] == 1]
testing_neg_dataset_df = testing_dataset_df[testing_dataset_df['label'] == 0]

In [10]:
# Convert ALL text rows to string, if not already string (this was causing issues which is why im now converting to string)
# Ignore the output warnings, or better yet, disable them. >:) (this is a bit of a hack, but it works)
training_pos_dataset_df['text'] = training_pos_dataset_df['text'].astype(str)
training_neg_dataset_df['text'] = training_neg_dataset_df['text'].astype(str)
testing_pos_dataset_df['text'] = testing_pos_dataset_df['text'].astype(str)
testing_neg_dataset_df['text'] = testing_neg_dataset_df['text'].astype(str)

/var/folders/lt/mn6whq3x1l106f5w78wq5x780000gn/T/ipykernel_11138/4058182508.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_pos_dataset_df['text'] = training_pos_dataset_df['text'].astype(str)
/var/folders/lt/mn6whq3x1l106f5w78wq5x780000gn/T/ipykernel_11138/4058182508.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_neg_dataset_df['text'] = training_neg_dataset_df['text'].astype(str)
/var/folders/lt/mn6whq3x1l106f5w78wq5x780000gn/T/ipykernel_11138/4058182508.py:5: SettingWithCopyWar

In [11]:
# Lowercase ALL sets
# Ignore the output warnings, or better yet, disable them. >:) (this is a bit of a hack, but it works)
training_pos_dataset_df.loc[:,'text'] = training_pos_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
training_neg_dataset_df.loc[:,'text'] = training_neg_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
testing_pos_dataset_df.loc[:,'text'] = testing_pos_dataset_df.loc[:,'text'].apply(lambda x: x.lower())
testing_neg_dataset_df.loc[:,'text'] = testing_neg_dataset_df.loc[:,'text'].apply(lambda x: x.lower())

/Users/austin/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [12]:
# Dataframe rows to lists
training_pos_dataset_list = training_pos_dataset_df['text'].tolist()
training_neg_dataset_list = training_neg_dataset_df['text'].tolist()
testing_pos_dataset_list = testing_pos_dataset_df['text'].tolist()
testing_neg_dataset_list = testing_neg_dataset_df['text'].tolist()

In [13]:
# Remove all non-alphabetical characters
training_pos_dataset_list_alpha_cleaned = []
training_neg_dataset_list_alpha_cleaned = []
testing_pos_dataset_list_alpha_cleaned = []
testing_neg_dataset_list_alpha_cleaned = []

for i in range(len(training_pos_dataset_list)):
    training_pos_dataset_list_alpha_cleaned.append(''.join([ch for ch in training_pos_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(training_neg_dataset_list)):
    training_neg_dataset_list_alpha_cleaned.append(''.join([ch for ch in training_neg_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(testing_pos_dataset_list)):
    testing_pos_dataset_list_alpha_cleaned.append(''.join([ch for ch in testing_pos_dataset_list[i] if ch.isalpha() or ch == ' ']))

for i in range(len(testing_neg_dataset_list)):
    testing_neg_dataset_list_alpha_cleaned.append(''.join([ch for ch in testing_neg_dataset_list[i] if ch.isalpha() or ch == ' ']))

# wow i cant believe that worked?!

In [14]:
# Tokenize each list
training_pos_dataset_list_alpha_cleaned_tokenized = []
training_neg_dataset_list_alpha_cleaned_tokenized = []
testing_pos_dataset_list_alpha_cleaned_tokenized = []
testing_neg_dataset_list_alpha_cleaned_tokenized = []

for i in range(len(training_pos_dataset_list_alpha_cleaned)):
    training_pos_dataset_list_alpha_cleaned_tokenized += training_pos_dataset_list_alpha_cleaned[i].split()

for i in range(len(training_neg_dataset_list_alpha_cleaned)):
    training_neg_dataset_list_alpha_cleaned_tokenized += training_neg_dataset_list_alpha_cleaned[i].split()

# for i in range(len(testing_pos_dataset_list_alpha_cleaned)):
#     testing_pos_dataset_list_alpha_cleaned_tokenized += testing_pos_dataset_list_alpha_cleaned[i].split()

# for i in range(len(testing_neg_dataset_list_alpha_cleaned)):
#     testing_neg_dataset_list_alpha_cleaned_tokenized += testing_neg_dataset_list_alpha_cleaned[i].split()

In [15]:
len(training_pos_dataset_list_alpha_cleaned_tokenized)

208351609

In [16]:
# Remove stopwords
training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed = []
stop_words = set(sw.words('english'))


for i in training_pos_dataset_list_alpha_cleaned_tokenized:
    if i not in stop_words:
        training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

for i in training_neg_dataset_list_alpha_cleaned_tokenized:
    if i not in stop_words:
        training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

# for i in testing_pos_dataset_list_alpha_cleaned_tokenized:
#     if i not in stop_words:
#         testing_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

# for i in testing_neg_dataset_list_alpha_cleaned_tokenized:
#     if i not in stop_words:
#         testing_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed.append(i)

In [17]:
len(training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed)

109774389

In [28]:
# Stemmed each set:

ps = PorterStemmer()

training_pos_dataset_list_alpha_cleaned_stemmed = []
training_neg_dataset_list_alpha_cleaned_stemmed = []
testing_pos_dataset_list_alpha_cleaned_stemmed = []
testing_neg_dataset_list_alpha_cleaned_stemmed = []

for i in range(len(training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    training_pos_dataset_list_alpha_cleaned_stemmed.append(ps.stem(training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i]))

for i in range(len(training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed)):
    training_neg_dataset_list_alpha_cleaned_stemmed.append(ps.stem(training_neg_dataset_list_alpha_cleaned_tokenized_stopwords_removed[i]))

KeyboardInterrupt: 

In [24]:
training_pos_dataset_list_alpha_cleaned_lemmatized


['lol',
 'love',
 'game',
 'wow',
 'didnt',
 'expect',
 'one',
 'best',
 'stori',
 'ever',
 'sad',
 'million',
 'doller',
 'game',
 'cant',
 'ever',
 'live',
 'game',
 'main',
 'charact',
 'rectangl',
 'spent',
 'countless',
 'night',
 'play',
 'game',
 'couldnt',
 'sleep',
 'mayb',
 'couldnt',
 'sleep',
 'play',
 'game',
 'p',
 'game',
 'scenario',
 'origin',
 'game',
 'well',
 'make',
 'even',
 'awesom',
 'basic',
 'your',
 'group',
 'four',
 'youv',
 'stick',
 'togeth',
 'surviv',
 'hoard',
 'zombi',
 'often',
 'run',
 'toward',
 'lot',
 'differ',
 'cool',
 'map',
 'choos',
 'also',
 'differ',
 'weapon',
 'depend',
 'style',
 'play',
 'even',
 'smash',
 'guitar',
 'want',
 'although',
 'id',
 'rather',
 'save',
 'guitar',
 'tbh',
 'love',
 'coop',
 'game',
 'enjoy',
 'kill',
 'thing',
 'need',
 'get',
 'game',
 'thank',
 'later',
 'high',
 'recommend',
 'download',
 'age',
 'empir',
 'today',
 'game',
 'goe',
 'freez',
 'second',
 'singl',
 'aswel',
 'campighn',
 'help',
 'wow',
 'e

In [22]:
training_pos_dataset_list_alpha_cleaned_tokenized_stopwords_removed

['lol',
 'love',
 'game',
 'wow',
 'didnt',
 'expect',
 'one',
 'best',
 'stories',
 'ever',
 'sad',
 'million',
 'doller',
 'games',
 'cant',
 'ever',
 'live',
 'game',
 'main',
 'character',
 'rectangle',
 'spent',
 'countless',
 'nights',
 'playing',
 'game',
 'couldnt',
 'sleep',
 'maybe',
 'couldnt',
 'sleep',
 'played',
 'game',
 'p',
 'game',
 'scenarios',
 'original',
 'game',
 'well',
 'makes',
 'even',
 'awesome',
 'basically',
 'youre',
 'group',
 'four',
 'youve',
 'stick',
 'together',
 'survive',
 'hoard',
 'zombies',
 'often',
 'running',
 'towards',
 'lots',
 'different',
 'cool',
 'maps',
 'choose',
 'also',
 'different',
 'weapons',
 'depending',
 'style',
 'play',
 'even',
 'smash',
 'guitar',
 'wanted',
 'although',
 'id',
 'rather',
 'save',
 'guitar',
 'tbh',
 'love',
 'coop',
 'games',
 'enjoy',
 'killing',
 'things',
 'need',
 'get',
 'game',
 'thank',
 'later',
 'highely',
 'recommended',
 'downloaded',
 'age',
 'empires',
 'today',
 'game',
 'goes',
 'freezes'

In [ ]:
len(testing_dataset_df)

In [ ]:
training_neg_dataset_df.head()

In [ ]:
training_dataset_df.head()

In [ ]:
tokens = [tokens.lower() for tokens in help ]

In [ ]:
print(tokens)

In [ ]:
stop_words = set(sw.words('english'))
filtered_corpus = []

for i in tokens:
    if i not in stop_words:
        filtered_corpus.append(i)

print( filtered_corpus)

In [ ]:
ps = PorterStemmer()

for w in filtered_corpus:
    print(w, " : ", ps.stem(w))